In [4]:
import sys
sys.path.append('../scraper')

from epl_table_scraper import WebScraper

scraper = WebScraper(url="https://understat.com/league/EPL")
scraper.load_page()
scraper.click_element("home-away2")

scraper is locating element...
element located:  <selenium.webdriver.remote.webelement.WebElement (session="a500e9f49742a4f9fc51d0bcc8fe6d19", element="f.0743CB4731B8DF42C946CA94FCB737E1.d.3BDB0074A8DA9E084EBF0A1EE8E77872.e.12")>
element html:  <input id="home-away2" type="radio" name="home-away" value="h">
